In [ ]:
!git clone https://github.com/xuebinqin/U-2-Net.git
!cd U-2-Net
!wget https://github.com/xuebinqin/U-2-Net/releases/download/v1.0/u2net.pth

In [ ]:
!git clone https://github.com/xuebinqin/U-2-Net.git

In [ ]:
import os
import sys
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from torch.autograd import Variable

sys.path.append('/content/U-2-Net/model')
from u2net import U2NET

# Load U2-Net model
model_dir = '/content/drive/MyDrive/u2net.pth'
net = U2NET(3, 1)
net.load_state_dict(torch.load(model_dir, map_location='cuda' if torch.cuda.is_available() else 'cpu'))
net.eval()

input_directory = '/content/drive/MyDrive/input2/'
output_directory = '/content/drive/MyDrive/output2/'

# Image transformations
transform = transforms.Compose([
    transforms.Resize((320, 320)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def remove_background(image_path, output_path):
    image = Image.open(image_path).convert('RGB')
    transformed_image = transform(image).unsqueeze(0)

    # Process image
    input_var = Variable(transformed_image)
    if torch.cuda.is_available():
        input_var = input_var.cuda()
        net.cuda()

    with torch.no_grad():
        d1, _, _, _, _, _, _ = net(input_var)

    # Normalize prediction to [0, 1]
    pred = d1[:, 0, :, :]
    pred = (pred - pred.min()) / (pred.max() - pred.min())
    pred_np = pred.squeeze().cpu().data.numpy()

    # Convert numpy array to image
    mask = Image.fromarray((pred_np * 255).astype(np.uint8))
    mask = mask.resize(image.size, resample=Image.BILINEAR)

    # Apply mask to the original image
    empty_img = Image.new("RGBA", image.size)
    output_img = Image.composite(image, empty_img, mask)


    output_img.save(output_path, format='PNG')

# Iterate through folders and files
i = 0
for root, dirs, files in os.walk(input_directory):
    for filename in files:
        if filename.lower().endswith(('.jpg', '.png', '.webp', '.jfif', '.gif')):
            input_path = os.path.join(root, filename)
            relative_path = os.path.relpath(root, input_directory)
            output_folder = os.path.join(output_directory, relative_path)
            os.makedirs(output_folder, exist_ok=True)
            output_path = os.path.join(output_folder, filename)
            remove_background(input_path, output_path)
            i += 1
            print(f'Image processing: {i} - Saved: {output_path}')

print("Processing complete")
